# **💡0️⃣ [ 아래 코드를 실행하여 조별 번호를 입력하시오.]💡**

*   1조: 박유진, 김서연, 최진
*   2조: 서민경, 이유진, 최범영
*   3조: 정은주, 김장환, 권진경
*   4조: 정진교, 장채은, 조성환
*   5조: 전승재, 신은호, 김윤희
*   6조: 정현서, 진경은, 이정민
*   7조: 이성지, 김내경



In [1]:
print("[예]조의 번호를 입력하시오: 1")
team_idx = int(input(">> 조의 번호를 입력하시오: "))
print(f">> {team_idx}조 입니다.")
team_idx = 'team'+str(team_idx)

[예]조의 번호를 입력하시오: 1
>> 5조 입니다.


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm

import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision.datasets.utils import download_url, extract_archive
import random
import matplotlib.pyplot as plt
import numpy as np
import os

%matplotlib inline
from torchinfo import summary

URL = "https://github.com/JanghunHyeon/AISW4202-Project/releases/download/v.1.1.0/project_dataset.zip"
ROOT = "./content/data"
ZIP_PATH = os.path.join(ROOT, "project_dataset.zip")
OUT_DIR  = os.path.join(ROOT, "project_dataset")

os.makedirs(ROOT, exist_ok=True)
download_url(URL, root=ROOT, filename="project_dataset.zip")
extract_archive(ZIP_PATH, OUT_DIR)

device = 'cuda' if torch.cuda.is_available() else 'cpu'


# Reproduce를 위한 Seed 고정
seed_id = 777
deterministic = True

random.seed(seed_id)
np.random.seed(seed_id)
torch.manual_seed(seed_id)
if device =='cuda':
    torch.cuda.manual_seed_all(seed_id)
if deterministic:
	torch.backends.cudnn.deterministic = True
	torch.backends.cudnn.benchmark = False

# **💡1️⃣ [ Design your custom model ]💡**
  ## 아래 코드를 수정하여 본인의 모델을 만들어 보세요.

In [ ]:
class SEBlock(nn.Module):
    """Squeeze-and-Excitation Block"""
    def __init__(self, channels, reduction=16):
        super(SEBlock, self).__init__()
        self.global_pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Linear(channels, channels // reduction)
        self.fc2 = nn.Linear(channels // reduction, channels)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        b, c, _, _ = x.size()
        # Squeeze
        se = self.global_pool(x).view(b, c)
        # Excitation
        se = self.fc1(se)
        se = self.relu(se)
        se = self.fc2(se)
        se = self.sigmoid(se).view(b, c, 1, 1)
        # Scale
        return x * se.expand_as(x)


class SeparableConv2d(nn.Module):
    """Depthwise Separable Convolution from Xception"""
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1):
        super(SeparableConv2d, self).__init__()
        self.depthwise = nn.Conv2d(in_channels, in_channels, kernel_size=kernel_size, 
                                 stride=stride, padding=padding, groups=in_channels)
        self.pointwise = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.depthwise(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.pointwise(x)
        x = self.bn2(x)
        return x


class XceptionBlock(nn.Module):
    """Modified Xception Block with Residual Connection and SE"""
    def __init__(self, in_channels, out_channels, stride=1, use_se=True):
        super(XceptionBlock, self).__init__()
        
        self.use_residual = (in_channels == out_channels and stride == 1)
        
        # Main path with separable convolutions
        self.sep_conv1 = SeparableConv2d(in_channels, out_channels, stride=1)
        self.sep_conv2 = SeparableConv2d(out_channels, out_channels, stride=stride)
        self.relu = nn.ReLU()
        
        # SE block
        self.use_se = use_se
        if use_se:
            self.se = SEBlock(out_channels)
        
        # Shortcut connection
        if not self.use_residual:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )
    
    def forward(self, x):
        residual = x
        
        # Main path
        out = self.sep_conv1(x)
        out = self.relu(out)
        out = self.sep_conv2(out)
        
        # SE attention
        if self.use_se:
            out = self.se(out)
        
        # Residual connection
        if self.use_residual:
            out = out + residual
        else:
            out = out + self.shortcut(residual)
        
        return self.relu(out)
class SEBlock(nn.Module):
    """Squeeze-and-Excitation Block"""
    def __init__(self, channels, reduction=16):
        super(SEBlock, self).__init__()
        self.global_pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Linear(channels, channels // reduction)
        self.fc2 = nn.Linear(channels // reduction, channels)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        b, c, _, _ = x.size()
        # Squeeze
        se = self.global_pool(x).view(b, c)
        # Excitation
        se = self.fc1(se)
        se = self.relu(se)
        se = self.fc2(se)
        se = self.sigmoid(se).view(b, c, 1, 1)
        # Scale
        return x * se.expand_as(x)


class AdvancedSeparableConv2d(nn.Module):
    """Advanced Depthwise Separable Convolution with multiple kernel sizes"""
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1,
                 use_multiple_kernels=True, expansion_factor=1):
        super(AdvancedSeparableConv2d, self).__init__()
        
        self.use_multiple_kernels = use_multiple_kernels
        expanded_channels = in_channels * expansion_factor
        
        if use_multiple_kernels:
            # Multi-scale depthwise convolution (like MobileNetV2 inspiration)
            self.depthwise_3x3 = nn.Conv2d(in_channels, expanded_channels//2, kernel_size=3, 
                                         stride=stride, padding=1, groups=in_channels)
            self.depthwise_5x5 = nn.Conv2d(in_channels, expanded_channels//2, kernel_size=5, 
                                         stride=stride, padding=2, groups=in_channels)
            self.bn_dw = nn.BatchNorm2d(expanded_channels)
        else:
            # Standard depthwise convolution
            self.depthwise = nn.Conv2d(in_channels, expanded_channels, kernel_size=kernel_size, 
                                     stride=stride, padding=padding, groups=in_channels)
            self.bn_dw = nn.BatchNorm2d(expanded_channels)
        
        # Pointwise convolution
        self.pointwise = nn.Conv2d(expanded_channels, out_channels, kernel_size=1)
        self.bn_pw = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        if self.use_multiple_kernels:
            # Multi-scale feature extraction
            dw_3x3 = self.depthwise_3x3(x)
            dw_5x5 = self.depthwise_5x5(x)
            x = torch.cat([dw_3x3, dw_5x5], dim=1)
        else:
            x = self.depthwise(x)
        
        x = self.bn_dw(x)
        x = self.relu(x)
        x = self.pointwise(x)
        x = self.bn_pw(x)
        return x


class MobileNetV2Block(nn.Module):
    """MobileNetV2 Inverted Residual Block with Depthwise Separable Conv"""
    def __init__(self, in_channels, out_channels, stride=1, expansion_factor=6):
        super(MobileNetV2Block, self).__init__()
        
        self.use_residual = (stride == 1 and in_channels == out_channels)
        expanded_channels = in_channels * expansion_factor
        
        # Expansion phase (1x1 conv)
        self.expand = nn.Sequential(
            nn.Conv2d(in_channels, expanded_channels, kernel_size=1),
            nn.BatchNorm2d(expanded_channels),
            nn.ReLU()
        ) if expansion_factor != 1 else nn.Identity()
        
        # Depthwise phase (3x3 depthwise conv)
        self.depthwise = nn.Sequential(
            nn.Conv2d(expanded_channels, expanded_channels, kernel_size=3, 
                     stride=stride, padding=1, groups=expanded_channels),
            nn.BatchNorm2d(expanded_channels),
            nn.ReLU()
        )
        
        # Pointwise phase (1x1 conv, no activation)
        self.pointwise = nn.Sequential(
            nn.Conv2d(expanded_channels, out_channels, kernel_size=1),
            nn.BatchNorm2d(out_channels)
        )
        
        # SE attention
        self.se = SEBlock(out_channels, reduction=8)
    
    def forward(self, x):
        residual = x
        
        # Expansion
        x = self.expand(x)
        
        # Depthwise
        x = self.depthwise(x)
        
        # Pointwise
        x = self.pointwise(x)
        
        # SE attention
        x = self.se(x)
        
        # Residual connection
        if self.use_residual:
            x = x + residual
        
        return x


class DeepSeparableBlock(nn.Module):
    """Deep Separable Block with multiple separable convolutions"""
    def __init__(self, in_channels, out_channels, num_layers=3, stride=1):
        super(DeepSeparableBlock, self).__init__()
        
        layers = []
        current_channels = in_channels
        
        for i in range(num_layers):
            # First layer handles stride, others use stride=1
            layer_stride = stride if i == 0 else 1
            layer_out_channels = out_channels if i == num_layers - 1 else current_channels
            
            layers.append(AdvancedSeparableConv2d(
                current_channels, layer_out_channels, 
                stride=layer_stride, use_multiple_kernels=(i % 2 == 0),
                expansion_factor=2 if i == 0 else 1
            ))
            layers.append(nn.ReLU())
            current_channels = layer_out_channels
        
        self.layers = nn.Sequential(*layers)
        
        # Skip connection
        self.use_skip = (in_channels == out_channels and stride == 1)
        if not self.use_skip and stride != 1:
            self.skip = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )
        elif not self.use_skip:
            self.skip = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1),
                nn.BatchNorm2d(out_channels)
            )
    
    def forward(self, x):
        residual = x
        out = self.layers(x)
        
        if self.use_skip:
            out = out + residual
        else:
            out = out + self.skip(residual)
        
        return out


class XceptionBlock(nn.Module):
    """Enhanced Xception Block with Advanced Separable Convolutions"""
    def __init__(self, in_channels, out_channels, stride=1, use_se=True):
        super(XceptionBlock, self).__init__()
        
        self.use_residual = (in_channels == out_channels and stride == 1)
        
        # Enhanced separable convolutions
        self.sep_conv1 = AdvancedSeparableConv2d(in_channels, out_channels, 
                                               use_multiple_kernels=True, 
                                               expansion_factor=2)
        self.sep_conv2 = AdvancedSeparableConv2d(out_channels, out_channels, 
                                               stride=stride,
                                               use_multiple_kernels=False,
                                               expansion_factor=1)
        self.relu = nn.ReLU()
        
        # SE block
        self.use_se = use_se
        if use_se:
            self.se = SEBlock(out_channels, reduction=8)
        
        # Shortcut connection
        if not self.use_residual:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )
    
    def forward(self, x):
        residual = x
        
        # Enhanced separable convolutions
        out = self.sep_conv1(x)
        out = self.relu(out)
        out = self.sep_conv2(out)
        
        # SE attention
        if self.use_se:
            out = self.se(out)
        
        # Residual connection
        if self.use_residual:
            out = out + residual
        else:
            out = out + self.shortcut(residual)
        
        return self.relu(out)
    
    def forward(self, x):
        residual = x
        
        # Main path
        out = self.sep_conv1(x)
        out = self.relu(out)
        out = self.sep_conv2(out)
        
        # SE attention
        if self.use_se:
            out = self.se(out)
        
        # Residual connection
        if self.use_residual:
            out = out + residual
        else:
            out = out + self.shortcut(residual)
        
        return self.relu(out)


class MyCustomModel(nn.Module):
    def __init__(self, num_classes=15, init_weights=True):
        super(MyCustomModel, self).__init__()
        
        # Entry flow - Initial convolution (48x48 -> 24x24)
        self.entry_conv = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        
        # Middle flow - Xception blocks with SE attention (24x24 -> 12x12 -> 6x6)
        self.xception_block1 = XceptionBlock(64, 128, stride=2, use_se=True)
        self.xception_block2 = XceptionBlock(128, 256, stride=2, use_se=True)
        self.xception_block3 = XceptionBlock(256, 512, stride=1, use_se=True)
        
        # Additional residual blocks for better feature extraction
        self.res_block1 = self._make_residual_block(512, 512)
        
        # Exit flow - Final feature extraction
        self.exit_conv = nn.Sequential(
            SeparableConv2d(512, 728, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            SeparableConv2d(728, 1024, kernel_size=3, stride=1, padding=1),
            nn.ReLU()
        )
        
        # SE block for final features
        self.final_se = SEBlock(1024, reduction=32)
        
        # Global average pooling and classifier
        self.global_pool = nn.AdaptiveAvgPool2d(1)
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(1024, num_classes)
        
        if init_weights:
            self._initialize_weights()
    
    def _make_residual_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels // 4, kernel_size=1),
            nn.BatchNorm2d(out_channels // 4),
            nn.ReLU(),
            nn.Conv2d(out_channels // 4, out_channels // 4, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels // 4),
            nn.ReLU(),
            nn.Conv2d(out_channels // 4, out_channels, kernel_size=1),
            nn.BatchNorm2d(out_channels),
        )
    
    def forward(self, x):
        # Entry flow
        x = self.entry_conv(x)  # 48x48 -> 24x24, channels: 3 -> 64
        
        # Middle flow with Xception blocks
        x = self.xception_block1(x)  # 24x24 -> 12x12, channels: 64 -> 128
        x = self.xception_block2(x)  # 12x12 -> 6x6, channels: 128 -> 256
        x = self.xception_block3(x)  # 6x6 -> 6x6, channels: 256 -> 512
        
        # Additional residual blocks with skip connections
        residual1 = x
        x = self.res_block1(x)
        x = x + residual1
        x = nn.ReLU()(x)
        
        # Exit flow
        x = self.exit_conv(x)  # channels: 512 -> 1024
        
        # Final SE attention
        x = self.final_se(x)
        
        # Global pooling and classification
        x = self.global_pool(x)  # 6x6 -> 1x1
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = self.fc(x)
        
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

In [19]:
model = MyCustomModel(num_classes=15).to(device)

inputs = torch.Tensor(1,3,48,48).to(device)
out = model(inputs)
print(out.shape)

AttributeError: 'MyCustomModel' object has no attribute 'xception_block4'

In [13]:
print(out)

tensor([[-0.0931, -0.1128,  0.1240,  0.0888,  0.0082,  0.0536,  0.3186, -0.0539,
          0.0479,  0.0663,  0.0693, -0.0390, -0.0170,  0.1689, -0.2925]],
       device='cuda:0', grad_fn=<AddmmBackward0>)


In [14]:
summary(model, (1, 3, 48, 48))

Layer (type:depth-idx)                   Output Shape              Param #
MyCustomModel                            [1, 15]                   --
├─Sequential: 1-1                        [1, 64, 24, 24]           --
│    └─Conv2d: 2-1                       [1, 32, 24, 24]           896
│    └─BatchNorm2d: 2-2                  [1, 32, 24, 24]           64
│    └─ReLU: 2-3                         [1, 32, 24, 24]           --
│    └─Conv2d: 2-4                       [1, 64, 24, 24]           18,496
│    └─BatchNorm2d: 2-5                  [1, 64, 24, 24]           128
│    └─ReLU: 2-6                         [1, 64, 24, 24]           --
├─XceptionBlock: 1-2                     [1, 128, 12, 12]          --
│    └─AdvancedSeparableConv2d: 2-7      [1, 128, 24, 24]          --
│    │    └─Conv2d: 3-1                  [1, 64, 24, 24]           640
│    │    └─Conv2d: 3-2                  [1, 64, 24, 24]           1,664
│    │    └─BatchNorm2d: 3-3             [1, 128, 24, 24]          256
│   

# **💡2️⃣[ Dataset Processing ]💡**

## Dataset for Final Project
- Large-scale RGB image dataset
  - Training set: about 75,000 images
  - Validation set: 900 images
  
- Dataset specification
  - image size: 48 x 48
  - RGB scale image (3 channel)
  - 15 classes

In [15]:
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std =[0.229, 0.224, 0.225]
    ),
])

transform_val = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std =[0.229, 0.224, 0.225]
    ),
])

data_dir = os.path.join(OUT_DIR, "data/ProjectDataset")
trainset = ImageFolder(os.path.join(data_dir, "train"), transform=transform_train)
valset = ImageFolder(os.path.join(data_dir, "val"), transform=transform_val)

trainloader = DataLoader(trainset, batch_size=256, shuffle=True, num_workers=6)
valloader = DataLoader(valset, batch_size=64, shuffle=False, num_workers=6)

# **💡3️⃣[ Train your custom model ]💡**


In [16]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.005)

lr_sche = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.9)

In [17]:
print(len(trainloader))
epochs = 50

for epoch in range(epochs):  # loop over the dataset multiple times
    model.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 50 == 49:    # print every 50 iterations
            print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / 50))
            running_loss = 0.0

    lr_sche.step()

    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for data in valloader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)

            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)

            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 1,200 validation images: %.3f %%' % (100 * correct / total))


print('Finished Training')

288
[1,    50] loss: 2.078
[1,   100] loss: 1.666
[1,   150] loss: 1.529
[1,   200] loss: 1.400
[1,   250] loss: 1.343
Accuracy of the network on the 1,200 validation images: 58.556 %
[2,    50] loss: 1.189
[2,   100] loss: 1.156
[2,   150] loss: 1.123
[2,   200] loss: 1.097
[2,   250] loss: 1.039
Accuracy of the network on the 1,200 validation images: 69.333 %
[3,    50] loss: 0.977
[3,   100] loss: 0.962
[3,   150] loss: 0.925
[3,   200] loss: 0.936
[3,   250] loss: 0.928
Accuracy of the network on the 1,200 validation images: 71.889 %
[4,    50] loss: 0.828
[4,   100] loss: 0.850
[4,   150] loss: 0.833
[4,   200] loss: 0.836
[4,   250] loss: 0.835
Accuracy of the network on the 1,200 validation images: 72.556 %
[5,    50] loss: 0.764
[5,   100] loss: 0.749
[5,   150] loss: 0.776
[5,   200] loss: 0.765
[5,   250] loss: 0.769
Accuracy of the network on the 1,200 validation images: 73.556 %
[6,    50] loss: 0.686
[6,   100] loss: 0.692
[6,   150] loss: 0.689
[6,   200] loss: 0.694
[6, 

KeyboardInterrupt: 

# 💡4️⃣ [Model Evaluation]💡
* Validation Set을 이용한 성능 검증

In [15]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in valloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)

        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)

        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 1,200 validation images: %.3f %%' % (100 * correct / total))

Accuracy of the network on the 1,200 validation images: 6.667 %


# 💡5️⃣ [Save Model]💡
1. 아래 코드를 동작하면 Google 인증 팝업이 뜨고 인증을 요청.
2. 인증후, Drive에 Final_Project/Model이라는 폴더가 생성됨.
3. 생성된 폴더 안에 **model_{team_idx}.pt** 라는 파일이 생성됨.
4. 생성된 파일과 코드를 다운받아 LMS에 제출




In [16]:
fname = f"model_{team_idx}.pt"

model.eval()
example_input = torch.randn(1,3,48,48).to(device)
traced_script = torch.jit.trace(model, example_input)
traced_script.save(fname)


# 구축한 데이터셋 Google Drive로 저장
# #from google.colab import drive
# import shutil

# # 1) Drive 마운트
# #drive.mount('/content/drive')
# data_dir = './content'
# out_dirs = './content/Final_Project/model'
# os.makedirs(out_dirs, exist_ok=True)

# shutil.copy(os.path.join(data_dir, fname), os.path.join(out_dirs, fname))
# print("Copied to Drive → MyDrive")